## Intro

This is inspired by 
Article (likas2001probability) Likas, A. Probability density estimation using artificial neural networks Computer physics communications, Elsevier, 2001, 135, 167-175

But rather than estimating the working with a network, we will instead work with its derivitive.
This will let us replace their integration with a derivative.

Note that this method only works for compact supports



They use the PDF is given by $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p) dz}$$
and in their case $h=N(x,p)$  a neural network with weight and bias parameters $p$.
Where $S$ is a compact support. (That means bounded)


But if instead we say $h=\frac{\partial N(x,p)}{\partial x}$,

then $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p)}=\dfrac{\frac{\partial N(x,p)}{\partial x}}{N(max(S),p) - N(min(S), p)}$$

The denominator is ofcourse more complex for non-1D values of S.


The loss function given is the negative log-likelihood of the set of training samples $X$
$$L(p) = -\sum_{\forall x \in X} ln(h(x,p))  + |X| ln(\int_S h(z,p) dx)$$

Which befomes:

$$L(p) = -\sum_{\forall x \in X} log(\frac{\partial N(x,p)}{\partial x})  + |X|(ln(N(max(S),p)-N(min(S),p)) dx$$

In [ ]:
using Plots
using IJulia

In [ ]:
using TensorFlow
using Distributions
using StatsBase

In [ ]:
using DensityEstimationML
function only(itr)
    state = start(itr)
    val,state = next(itr, state)
    @assert(done(itr,state))
    return val
end

In [4]:
"""
Function returning a function that will display a running plot.
WARNING: Introducting or removing any variables is not supported.
And will silently error.
"""
function running_plot()
    epochs = Int[]
    record = Dict()
    function inner(epoch, vars::Associative)
        for (var, values) in vars
            value = only(values) #Incase it was an array
            past = get!(record, var) do
                typeof(value)[]
            end
            push!(past, value)
        end
        push!(epochs, epoch)
        
        IJulia.clear_output(true)
        plot(epochs, hcat(values(record)...); label=hcat(keys(vars)...), layout=length(vars)) |> IJulia.display       
    end
end


running_plot

In [5]:
function demonstration_plot(est, dataset, data, args...; kwargs...)
    X = minimum(approximate_support(dataset)) : 0.01 : maximum(approximate_support(dataset))
    @show typeof(data)
    println("True loglikelihood      = $(loglikelihood(dataset, data))")
    println("Estimated loglikelihood = $(loglikelihood(est, data))")
    plot([X], [pdf(est,X), data],
        #xlims= approximate_support(dataset),
        xlims= (first(X), last(X)),
        seriestype = [:path :histogram],
        layout=(2,1),
        legend=false,
        nbins=[1  length(data)÷10],
        args...; kwargs...
    )
end

demonstration_plot (generic function with 1 method)

In [6]:
function demo(dataset, layers, epochs=20_000; max_conditioning_epochs=2000)
    data = original_sample(dataset)
    @show loglikelihood(dataset, data)
    est = NeuralDensityEstimator(layers, approximate_support(dataset))

    condition!(est; max_epochs = max_conditioning_epochs)
    println("Conditioning Done")
    Plots.gr()
    fit!(est, data; epochs=epochs, callback=running_plot())
    println("Fitting Done")
    plotly()
    demonstration_plot(est, dataset, data) |> IJulia.display
    
    est    
end

demo (generic function with 2 methods)

In [ ]:
est=demo(GenerateDatasets.Likas1(), [64, 64, 256], 1_000)

In [ ]:
loglikelihood(GenerateDatasets.Likas1(), [2.0])

In [ ]:
GenerateDatasets.Likas1()

In [ ]:
sort(component_weights(est))

In [ ]:
demo(GenerateDatasets.Likas2(), [64,64], 20_000)

In [ ]:
demo(GenerateDatasets.MagdonIsmailAndAtiya(), [32], 10_000)

In [ ]:
demo(Arcsine(1,4), [64,64], 20_000)

In [10]:
dataset = GenerateDatasets.Likas3()
data = original_sample(dataset)

2×5000 Array{Float64,2}:
 0.168274  0.0164784  0.0360932  0.0139004  …  0.16162   0.168491  0.00861437
 0.100672  0.128224   0.166696   0.0778806     0.142952  0.142529  0.0350145 

In [13]:
@show loglikelihood(dataset, data)

loglikelihood(dataset, data) = 16094.379124341185


16094.379124341185

In [15]:

est = NeuralDensityEstimator([64], (-0.1, -0.1),(0.3,0.3))

2017-09-20 14:18:33.536439: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2017-09-20 14:18:33.536470: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2017-09-20 14:18:33.536476: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2017-09-20 14:18:33.731465: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:893] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2017-09-20 14:18:33.731991: I tensorflow/core/common_runtime/gpu/gpu_device.cc:940] Found device 0 with propert

DensityEstimationML.NeuralDensityEstimator{2}(Session(Ptr{Void} @0x00007f7af9fef350), <Tensor Group:1 shape=unknown dtype=Any>, <Tensor Group_2:1 shape=unknown dtype=Any>, <Tensor t:1 shape=(2, ?) dtype=Float32>, <Tensor pdf:1 shape=unknown dtype=Float64>)

In [16]:

condition!(est)
println("Conditioning Done")
fit!(est, data; epochs=10_000)
println("Fitting Done")

(ii, ysmin, ysmax, condition_loss) = (1, [1.00279], [1.00279], [0.994445])
(ii, ysmin, ysmax, condition_loss) = (51, [1.15626], [1.15652], [0.73588])
(ii, ysmin, ysmax, condition_loss) = (101, [1.53301], [1.53886], [0.496748])
(ii, ysmin, ysmax, condition_loss) = (151, [1.50044], [1.51408], [0.486552])
(ii, ysmin, ysmax, condition_loss) = (201, [1.50034], [1.52536], [0.475627])
(ii, ysmin, ysmax, condition_loss) = (251, [1.50031], [1.54024], [0.461693])
(ii, ysmin, ysmax, condition_loss) = (301, [1.49963], [1.55843], [0.444619])
(ii, ysmin, ysmax, condition_loss) = (351, [1.49799], [1.58022], [0.424207])
(ii, ysmin, ysmax, condition_loss) = (401, [1.49489], [1.60586], [0.400265])
(ii, ysmin, ysmax, condition_loss) = (451, [1.48971], [1.63536], [0.372773])
(ii, ysmin, ysmax, condition_loss) = (501, [1.48173], [1.6683], [0.34209])
(ii, ysmin, ysmax, condition_loss) = (551, [1.47035], [1.70363], [0.309066])
(ii, ysmin, ysmax, condition_loss) = (601, [1.45528], [1.73986], [0.274952])
(ii, 

LoadError: [91mDimensionMismatch("Failed to set  input \"t:0\",  as Tensors have incompatiable shapes (TensorShape[2, ?] vs TensorShape[5000, 2])")[39m

In [ ]:
gr =est.sess.graph
run(est.sess, exp(6*gr["W_2"].^2 + gr["b_2"]))

In [ ]:
collect(keys(est.sess.graph))